In [90]:
# In the context of poker, if we are close to the end of the match, does it make sense to be more aggresive?


from random import random
from tqdm.notebook import tqdm
import numpy as np


num_matches = 100_000
chips_per_hand = 4 # amount you win if you both raise. This should be at least 4
blind_amount = 2 # amount you win if you steal blinds

In [99]:
def optimal_raise_percentage(blind_size, raise_size):
    for a in np.linspace(0,1,201):
        optimal_percentage = True
        for b in np.linspace(0,1,101):
            if a <= b:
                EV = a*(1-b)*blind_size - (1-a)*b*blind_size + a*(b-a)*raise_size
            else:
                EV = a*(1-b)*blind_size - (1-a)*b*blind_size + -b*(a-b)*raise_size
            if EV < 0: 
                optimal_percentage = False
                break
        if optimal_percentage:
            print(f'The optimal raise percentage for these blind/raise amounts is {a}')
            
optimal_raise_percentage(2,4)

The optimal raise percentage for these blind/raise amounts is 0.5
The optimal raise percentage for these blind/raise amounts is 0.505
The optimal raise percentage for these blind/raise amounts is 0.51


In [ ]:
def doesnt_matter(chips, hand_value):
    # Bulgaria poker champion's strategy. The stage of the game doesn't matter.
    return hand_value >= 0.505

def punishing_strategy(chips, hand_value):
    # Henry's strategy. Supposedly, we'd like to punish weaker opponents.
    if chips >= 180: 
        return hand_value > 0.405
    return hand_value > 0.505

def sample_game(player1, player2):
    chips1, chips2 = 100, 100
    while (chips1 > 0) and (chips2 > 0):
        hand1, hand2 = random(), random()
        raise1, raise2 = player1(chips1, hand1), player2(chips2, hand2)
        if raise1 and raise2: # both raise
            if hand1 > hand2:
                chips1, chips2 = chips1 + chips_per_hand, chips2 - chips_per_hand
            else:
                chips1, chips2 = chips1 - chips_per_hand, chips2 + chips_per_hand
        if raise1: # player1 steals blinds
            chips1, chips2 = chips1 + blind_amount, chips2 - blind_amount
        if raise2: # player2 steals blinds
            chips1, chips2 = chips1 - blind_amount, chips2 + blind_amount
    return chips2 <= 0
            
    
won_punish = 0
for _ in tqdm(range(num_matches)):
    won_punish += sample_game(punishing_strategy, doesnt_matter)
print(f'Punishing strategy won in {won_punish/num_matches*100}% of games')

  0%|          | 0/100000 [00:00<?, ?it/s]